[View in Colaboratory](https://colab.research.google.com/github/AwalDeep/SpeechRecognition/blob/master/speech.ipynb)

# Load libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import tensorflow  as tf
from tensorflow import keras
#import cupy as cp

from sklearn.preprocessing import normalize


In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Load data

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
cd gdrive/My\ Drive/

/content/gdrive/My Drive


In [5]:
cd HW1/

/content/gdrive/My Drive/HW1


In [0]:
#train_data=np.load("train.npy",encoding='bytes')
#train_labels=np.load("train_labels.npy",encoding='bytes')

In [0]:
dev_train=np.load("dev.npy",encoding='bytes')
dev_labels=np.load("dev_labels.npy",encoding='bytes')

# Adding context 

In [0]:
def context(utter,window):
  """Inputs : utterance and window size(multiples of 20)"""
  c=int(np.ceil(window/40))
  if window%40!=0:
    left=np.append(utter[-c:,:int(window%40)].reshape(c,-1),utter[:-c,:int(window%40)],axis=0)
    right=np.append(utter[c:,int(window%40):],utter[:c,:int(window%40)].reshape(c,-1),axis=0)
  else:
    
    left=np.append(utter[-c,:window].reshape(c,-1),utter[:-c,:int(window/c)],axis=0)
    right=np.append(utter[c:,:],utter[c,:window].reshape(c,-1),axis=0)
  #left.shape,right.shape
  new=np.hstack((left,utter,right))[c:-c,:]
  return new

In [0]:
#context on train data

#for i in range(len(train_data)):
#  train_data[i]=context(train_data[i],40)
#  #removing extra labels
#  train_labels[i]=train_labels[i][1:-1]


In [0]:
#context on dev data
for i in range(len(dev_train)):
  dev_train[i]=context(dev_train[i],40)
  #removing extra labels
  dev_labels[i]=dev_labels[i][1:-1]

# Concatenate and Normalize data

In [0]:
#data=np.concatenate(train_data)
#labels=np.concatenate(train_labels)
dev_data=np.concatenate(dev_train)
dev_labels=np.concatenate(dev_labels)


In [0]:
shuffle_list=np.arange(len(dev_data))
np.random.shuffle(shuffle_list)
np.random.shuffle(shuffle_list)

In [0]:
# randomly shuffle data

dev_data=dev_data[shuffle_list,:]
dev_labels=dev_labels[shuffle_list]

In [86]:
#dev_data=tf.data.Dataset.shuffle(dev_data,buffer_size=600,seed=100)
#dev_labels=tf.data.Dataset.shuffle(dev_labels,buffer_size=600,seed=100)


(667088, 80)

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(dev_data,dev_labels,test_size=0.15)

In [0]:
#normalize train data
#data=normalize(data)
#normalise dev data
dev_data=normalize(x_train)
test_data=normalize(x_test)

# Model


In [0]:
#data=cp.array(data)
#labels=cp.array(labels)

In [0]:
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(120,)),
    keras.layers.Dense(700, activation=tf.nn.leaky_relu),
    #keras.layers.Dropout(0.01),
    keras.layers.Dense(450, activation=tf.nn.leaky_relu),
    #keras.layers.Dropout(0.1),
    keras.layers.Dense(280, activation=tf.nn.leaky_relu),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(220),
    #, activation=tf.nn.leaky_relu),
    #keras.layers.Dropout(0.1),
    keras.layers.Dense(138, activation=tf.nn.softmax)
    
])

In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(), 
            loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [67]:
#batch_list=[510]
#for i in batch_list:

model.fit(dev_data,y_train,batch_size=500,epochs=10)
 

Epoch 1/10
567024/567024 [==============================] - 11s 20us/step - loss: 2.7722 - acc: 0.2994
Epoch 2/10
567024/567024 [==============================] - 10s 18us/step - loss: 2.4449 - acc: 0.3676
Epoch 3/10
567024/567024 [==============================] - 10s 18us/step - loss: 2.3368 - acc: 0.3905
Epoch 4/10
567024/567024 [==============================] - 10s 18us/step - loss: 2.2693 - acc: 0.4065
Epoch 5/10
567024/567024 [==============================] - 10s 18us/step - loss: 2.2195 - acc: 0.4174
Epoch 6/10
567024/567024 [==============================] - 10s 18us/step - loss: 2.1787 - acc: 0.4259
Epoch 7/10
567024/567024 [==============================] - 10s 17us/step - loss: 2.1435 - acc: 0.4338
Epoch 8/10
567024/567024 [==============================] - 10s 17us/step - loss: 2.1127 - acc: 0.4409
Epoch 9/10
567024/567024 [==============================] - 10s 17us/step - loss: 2.0875 - acc: 0.4461
Epoch 10/10
567024/567024 [==============================] - 10s 17us/ste

In [68]:
model.evaluate(test_data,y_test)

100064/100064 [==============================] - 11s 107us/step


[2.104687339244729, 0.44654421170450914]

In [0]:
#!cat /proc/meminfo
#!apt -y -q install cuda-libraries-dev-9-2
#!pip install -q cupy-cuda92 chainer